# The scripts for testing

In [ ]:
# Create the stamp for today's date
from datetime import datetime
today = str(datetime.now().strftime("%Y%m%d"))
print(today)

## Run LD-SDA / LD-BD Benchmarks
This notebook runs the 4 model benchmarks (toy, small-batch, CSTR, column) for both `gdpopt.ldsda` and `gdpopt.ldbd` with a shared configuration, and saves results under `LD_benchmark/results/<YYYYMMDD>/` (summary CSV, traj CSV, solver log).

In [ ]:
from pathlib import Path
import sys

# Locate LD_benchmark root and put it on sys.path so we can `import workflow.*`
cwd = Path.cwd().resolve()

def _find_ld_root(start: Path) -> Path:
    if (start / "models").is_dir() and (start / "results").is_dir():
        return start
    if (start / "LD_benchmark" / "models").is_dir():
        return start / "LD_benchmark"
    for parent in [start] + list(start.parents):
        if (parent / "models").is_dir() and (parent / "results").is_dir():
            return parent
        if (parent / "LD_benchmark" / "models").is_dir():
            return parent / "LD_benchmark"
    raise FileNotFoundError(f"Cannot find LD_benchmark root from {start}")

ld_root = _find_ld_root(cwd)
if str(ld_root) not in sys.path:
    sys.path.insert(0, str(ld_root))

# Also add local Pyomo source tree if present (repo layout: <workspace>/pyomo/pyomo/...)
pyomo_src = ld_root.parent / "pyomo"
if (pyomo_src / "pyomo").is_dir() and str(pyomo_src) not in sys.path:
    sys.path.insert(0, str(pyomo_src))

results_day_dir = ld_root / "results" / today
results_day_dir.mkdir(parents=True, exist_ok=True)
print("LD_benchmark root:", ld_root)
print("Pyomo src:", pyomo_src if (pyomo_src / "pyomo").is_dir() else "(not found)")
print("Results dir:", results_day_dir)

In [ ]:
from workflow.benchmark_suite import CommonConfig, run_all

# Common configuration shared by LDSDA and LDBD
common = CommonConfig(
    tee=False,
    time_limit=900,
    direction_norm="Linf",
    nlp_solver="ipopt",
    mip_solver="gurobi",
    separation_solver="gurobi",
)

# Required CSTR sizes
cstr_NT_list = [5, 10, 15, 20, 25, 30]

# Column random init: set to None to run ALL keys in models/column/column_initial_test.py
# Default here is a small subset to keep the benchmark runtime manageable.
column_keys = list(range(1, 11))

run_all(
    date_results_dir=results_day_dir,
    common=common,
    cstr_NT_list=cstr_NT_list,
    column_keys=column_keys,
 )

print("Done. Results written under:", results_day_dir)

## Testing Scripts
* Only test LD-SDA and LD-BD
